the purpose of this file is to create a csv file for each exome file, with the number of reads/not reads in each position

In [17]:
import pandas as pd
import os
from IPython.display import clear_output
from functools import reduce

In [12]:
FILES_LIST = ["/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ab_1..534.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ad_1..547.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ae_1..524.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ab_1..2641.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_aa_1..536.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ab_1..532.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ac_1..521.g.vcf.gz",]
CURR_FILE = 0
BATCH_SIZE = 10000
BLANK_CELL_PREFIX = "\./\."
BLANK_VAL = "."

In [ ]:
count = 0
dfs = []

In [ ]:

df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
while len(df) > 10:
    clear_output(wait=True)
    print(count)
    df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
    df["blank_sum"] = df.eq(BLANK_VAL).sum(axis=1)
    dfs.append(df[["#CHROM",	"POS",	"ID",	"REF",	"ALT",	"QUAL",	"FILTER",	"INFO",	"FORMAT","blank_sum"]])
    count += 1


693


In [22]:
len(dfs)

545

In [30]:
dfs[-1]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,blank_sum
0,chr2,124417414,.,C,<*>,0.0,.,END=124417414,GT:GQ:MIN_DP:MED_DP:PL,513
1,chr2,124417415,.,C,<*>,0.0,.,END=124417621,GT:GQ:MIN_DP:MED_DP:PL,511
2,chr2,124417416,.,T,<*>,0.0,.,END=124417422,GT:GQ:MIN_DP:MED_DP:PL,515
3,chr2,124417417,.,C,<*>,0.0,.,END=124417417,GT:GQ:MIN_DP:MED_DP:PL,515
4,chr2,124417418,.,A,<*>,0.0,.,END=124417625,GT:GQ:MIN_DP:MED_DP:PL,514
5,chr2,124417419,.,C,<*>,0.0,.,END=124417622,GT:GQ:MIN_DP:MED_DP:PL,516
6,chr2,124417420,.,T,<*>,0.0,.,END=124417420,GT:GQ:MIN_DP:MED_DP:PL,516
7,chr2,124417421,.,G,<*>,0.0,.,END=124417612,GT:GQ:MIN_DP:MED_DP:PL,516
8,chr2,124417422,.,C,<*>,0.0,.,END=124417422,GT:GQ:MIN_DP:MED_DP:PL,514
9,chr2,124417423,.,C,<*>,0.0,.,END=124417451,GT:GQ:MIN_DP:MED_DP:PL,514


## script

In [55]:
res = int(os.popen("zcat /sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz | head -n 12 | wc -l").read())

12

In [47]:
# part1 - in jupyter

# find the file length, 
# FILE_LENGTH = int(os.popen("zcat /sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz | wc -l").read())
FILE_LENGTH = 51049008

# part2 - master script
# find the number of instances and my instance.
SKIP = 3381
BATCH_SIZE = 10000

NUMBER_OF_INSTANCES = 200
MY_INSTANCE = 200

# find my part- which rows do I need to read
my_start = int(SKIP + (FILE_LENGTH - SKIP) * (MY_INSTANCE / (NUMBER_OF_INSTANCES + 1))) + 1
my_end = int(SKIP + (FILE_LENGTH - SKIP) * ((MY_INSTANCE+1) / (NUMBER_OF_INSTANCES + 1)))
count = 0
# df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(SKIP)) + list(range(my_start,count*BATCH_SIZE + my_start)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
while count*BATCH_SIZE + my_start < my_end:
    curr = count*BATCH_SIZE + my_start
    print(curr)
    count += 1
print(my_end)
# read these rows in jumps of 100,000 and append to list
# combine this list into csv

50795050
50805050
50815050
50825050
50835050
50845050
50855050
50865050
50875050
50885050
50895050
50905050
50915050
50925050
50935050
50945050
50955050
50965050
50975050
50985050
50995050
51005050
51015050
51025050
51035050
51045050
51049008
